Intro) Dans les jeux Pokemon, les pokémons sont regroupés dans ce que l'on appelle des "groupes d'oeufs". Ces groupes déterminent quels pokemons peuvent se reproduire entre eux et sont par conséquent similaire à la notion de "genre animal". Par exemple, le lion et le tigre font parti du même genre et peuve ainsi donner naissance à des hybrides.

Le but de ce projet est d'essayer de construire un modèle de prediction de l'appartenance ou non d'un pokémon aux différents groupes d'oeufs (CLASSIFICATION MULTILABEL)

Dans ce notebook,on extrait des features des séries temporelles via le package tsfresh. Ces features serviront en tant que variables prédictives pour le modèle final.

In [7]:
# importation des librairies
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np 
import tsfresh as ts 
import pandas as pd 
from tsfresh import select_features
import os 
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tqdm import tqdm

On va créer des features à partir des séries temporelles. Pour cela on utilise la librairie tsfresh avec comme features :

- la longueur de la série temporelle
- la moyenne
- la médiane
- le max
- le min
- l'energie du signal en valeur absolu
- le nombre de points en dessous de la moyenne
- Est ce que le signal est stationnaire (binaire)
- Le nombre de pics dans le signal sur une fenêtre de 15 points en utilisant les wavelets pour débruiter
- Le nombre de pics sur une fenêtre de 30 points
- Le nombre de points compris entre 0 et 1 s
- L'entropy du signal (https://en.wikipedia.org/wiki/Sample_entropy)
- La localisation du premier max
- La localisation du premier min
- La moyenne de la dérivée seconde
- La somme des points
- La variance

In [ ]:
df=pd.DataFrame()
i = 0
for numero_cluster in tqdm(range(1,19)):
    cluster_csv_folder = os.listdir(f"./CLASSES/{numero_cluster}/CSV")
    for csv in cluster_csv_folder:
        time_serie = pd.read_csv(f"./CLASSES/{numero_cluster}/CSV/{csv}")
        time_serie["time_in_seconds"] = time_serie.index
        time_serie["id"] = [str(i)]*len(time_serie)
        res = False
        fc_parameters = {
        "length": None,
        "mean":None,
        "maximum":None,
        "minimum":None,
        "abs_energy":None,
        "count_below_mean":None,
        "augmented_dickey_fuller":[{"attr":"pvalue","y":"AIC"}],
        "number_cwt_peaks":[{"n":15}],
        "number_peaks":[{"n":30}],
        "range_count":[{"min":0,"max":1}],
        "sample_entropy":None,
        "first_location_of_minimum":None,
        "first_location_of_maximum":None,
        "mean_second_derivative_central":None,
        "median":None,
        "standard_deviation":None,
        "sum_values":None,
        "variance":None,

        }
        settings = ComprehensiveFCParameters()
        df = df.append(ts.extract_features(time_serie,column_id="id", column_sort="time_in_seconds",default_fc_parameters=fc_parameters))
        i = i+1
df.head()

In [9]:
#on sauvegarde les resultats
df.to_csv("features.csv",index=False)